In [13]:
from faker import Faker
import random
import sqlite3
import pandas as pd

fake = Faker()

# Create a database connection
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Create Customers table
cursor.execute('''CREATE TABLE Customers (
                    customer_id INTEGER PRIMARY KEY,
                    name TEXT,
                    shipping_addr TEXT,
                    contact_number TEXT,
                    current_flag INTEGER DEFAULT 1
                )''')

# Create Orders table
cursor.execute('''CREATE TABLE Orders (
                    order_id INTEGER PRIMARY KEY,
                    customer_id INTEGER,
                    order_date TEXT,
                    total_amount REAL NOT NULL,
                    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
                )''')

# Create Order Items table
cursor.execute('''CREATE TABLE OrderItems (
                    item_id INTEGER PRIMARY KEY,
                    order_id INTEGER,
                    product_id INTEGER,
                    quantity INTEGER,
                    price REAL,
                    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
                    FOREIGN KEY (product_id) REFERENCES Products(product_id)
                )''')

# Create Products table
cursor.execute('''CREATE TABLE Products (
                    product_id INTEGER PRIMARY KEY,
                    name TEXT,
                    description TEXT
                )''')

# Generate customers
for _ in range(10):
    name = fake.name()
    shipping_addr = fake.address().replace("\n", ", ")
    contact_number = fake.phone_number()
    cursor.execute('''INSERT INTO Customers (name, shipping_addr, contact_number)
                      VALUES (?, ?, ?)''', (name, shipping_addr, contact_number))
    conn.commit()

# Generate products
for _ in range(10):
    name = fake.random_element(elements=('iPhone 13', 'iPad', 'MacBook Air', 'iphone 14', 'Apple Watch', 'AirPods', 'Apple TV', 'MacBook pro', 'iPod', 'Beats'))
    description = fake.sentence(nb_words=6)
    cursor.execute('''INSERT INTO Products (name, description)
                      VALUES (?, ?)''', (name, description))
    conn.commit()

# Generate orders and order items
for _ in range(100):
    customer_id = random.randint(1, 10)
    order_date = fake.date_between(start_date='-2y', end_date='today')
    total_amount = round(random.uniform(10, 1000), 2)
    cursor.execute('''INSERT INTO Orders (customer_id, order_date, total_amount)
                      VALUES (?, ?, ?)''', (customer_id, order_date, total_amount))
    order_id = cursor.lastrowid

    # Generate order items
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        product_id = random.randint(1, 10)
        quantity = random.randint(1, 10)
        price = round(random.uniform(10, 100), 2)
        cursor.execute('''INSERT INTO OrderItems (order_id, product_id, quantity, price)
                          VALUES (?, ?, ?, ?)''', (order_id, product_id, quantity, price))

    conn.commit()

# Close the database connection
conn.close()

In [14]:
# Create a database connection
conn = sqlite3.connect('ecommerce.db')
cursor = conn.cursor()

# Get orders in dataframe
cursor.execute("SELECT * FROM Orders")
orders = cursor.fetchall()
orders_df = pd.DataFrame(orders, columns=[column[0] for column in cursor.description])

# Get Customers in dataframe
cursor.execute("SELECT * FROM Customers")
customers = cursor.fetchall()
customers_df = pd.DataFrame(customers, columns=[column[0] for column in cursor.description])

# Get Products in dataframe
cursor.execute("SELECT * FROM Products")
products = cursor.fetchall()
products_df = pd.DataFrame(products, columns=[column[0] for column in cursor.description])

# Get OrderItems in dataframe
cursor.execute("SELECT * FROM OrderItems")
order_items = cursor.fetchall()
order_items_df = pd.DataFrame(order_items, columns=[column[0] for column in cursor.description])

# Close the database connection
conn.close()

In [15]:
orders_df.head()

,order_id,customer_id,order_date,total_amount
0,1,3,2021-12-26,749.26
1,2,9,2023-05-27,784.38
2,3,3,2022-02-12,10.21
3,4,7,2021-12-19,367.94
4,5,5,2022-06-21,461.62


In [16]:
customers_df.head()

,customer_id,name,shipping_addr,contact_number,current_flag
0,1,Pamela Skinner DDS,"8142 Christian Trace, Nguyenburgh, UT 90416",(980)976-1876x65084,1
1,2,Laurie Hamilton,"279 Ward Radial Apt. 443, New David, KS 05994",306-290-5316,1
2,3,Tim Clayton,"9227 Brown Island Apt. 292, Loriburgh, DC 00693",916.468.5834x674,1
3,4,Roberta Brown,"80050 Mejia Rapids, New Joseph, IN 60087",618-297-0878,1
4,5,Jordan Weeks,"USCGC Rosario, FPO AP 87171",(664)828-8845,1


In [23]:
products_df.head()

,product_id,name,description
0,1,iPod,Dog boy him food investment high.
1,2,iphone 14,Must look such bad movement world phone across.
2,3,Beats,Into contain sit ever.
3,4,Apple Watch,Phone carry have consumer hour its speak page.
4,5,iPhone 13,Dog remember situation recent.


In [18]:
order_items_df.head()

,item_id,order_id,product_id,quantity,price
0,1,1,9,5,78.30
1,2,1,8,4,50.21
2,3,2,9,9,50.70
3,4,3,7,6,69.83
4,5,3,9,5,13.19
